# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [1]:
%pip install --upgrade --quiet langchain langchain-community langchain_openai

Note: you may need to restart the kernel to use updated packages.


If you'd like to use LangSmith, uncomment the below:

In [2]:
import getpass
import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](https://python.langchain.com/docs/integrations/chat), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](https://python.langchain.com/docs/integrations/chat/ollama).

In [3]:
from langchain_community.llms import Ollama

model = Ollama(model="phi3")

C:\Users\adria\AppData\Local\Temp\ipykernel_12948\1427064109.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  model = Ollama(model="phi3")



#  How to Install and Run Ollama with the Phi-3 Model

This guide walks you through installing **Ollama** and running the **Phi-3** model on Windows, macOS, and Linux.

---

## Windows

1. **Download Ollama for Windows**  
   Go to: [https://ollama.com/download](https://ollama.com/download)  
   Download and run the installer.

2. **Verify Installation**  
   Open **Command Prompt** and type:
   ```bash
   ollama --version
   ```

3. **Run the Phi-3 Model**  
   In the same terminal:
   ```bash
   ollama run phi3
   ```

4. **If you get a CUDA error (GPU memory issue)**  
   Run Ollama in **CPU mode**:
   ```bash
   set OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  macOS

1. **Install via Homebrew**  
   Open the Terminal and run:
   ```bash
   brew install ollama
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Linux

1. **Install Ollama**  
   Open a terminal and run:
   ```bash
   curl -fsSL https://ollama.com/install.sh | sh
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Notes

- The first time you run `ollama run phi3`, it will **download the model**, so make sure you’re connected to the internet.
- Once downloaded, it works **offline**.
- Keep the terminal open and running in the background while using Ollama from your code or notebook.


## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](https://python.langchain.com/docs/how_to/custom_tools/).

In [4]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [5]:
multiply.invoke({"x": 4, "y": 5})

20.0

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [6]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


In [7]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [8]:
chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

{
  "name": "add",
  "arguments": {
    "x": 3,
    "y": 1132
  }
}


## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [9]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'x': 13, 'y': 4}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke 
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [11]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!

In [12]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [13]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})

53.83784653

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [14]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).

In [15]:
from langchain_core.prompts import ChatPromptTemplate

tool_example_1 = {
    "tool": "multiply",
    "args": {"a": 3, "b": 4},
}

tool_example_2 = {
    "tool": "multiply",
    "args": {"a": 13, "b": 2.5},
}

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
     You are a helpful assistant with access to tools. 
     Always respond using a JSON object with:
     - "tool": <tool_name>
     - "args": <argument dictionary>

     --- EXAMPLES ---
     USER: What is 3 times 4?
     ASSISTANT: ```json
     { "tool": "multiply", "args": {"a": 3, "b": 4} }
     ```

     USER: multiply 13 by 2.5
     ASSISTANT: ```json
     { "tool": "multiply", "args": {"a": 13, "b": 2.5} }
     ```

     --- END EXAMPLES ---

     Follow the same JSON format. No extra text.
     """
    ),
    ("human", "{input}")
])


In [16]:
from langchain_core.runnables import Runnable, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

def safe_parse(output):
    try:
        return parser.parse(output)
    except Exception as e:
        # If parsing fails, ask model to correct its JSON
        correction_prompt = f"""
        The previous output was invalid JSON.
        ERROR: {str(e)}

        Please correct the output. Only return VALID JSON for a tool call.
        Original output:
        {output}
        """
        fixed = model.invoke(correction_prompt).content
        return parser.parse(fixed)

safe_parser = RunnableLambda(safe_parse)


In [17]:
chain = (
    prompt
    | model
    | safe_parser
    | RunnablePassthrough.assign(output=invoke_tool)
)


In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser

# FEW-SHOT EXAMPLES
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a tool-using assistant. Always provide JSON.
    Here are correct examples:

    USER: What is 3 times 4?
    ASSISTANT:
    { "tool": "multiply", "args": {"a": 3, "b": 4} }

    USER: multiply 13 by 2.5
    ASSISTANT:
    { "tool": "multiply", "args": {"a": 13, "b": 2.5} }

    Follow this format.
    """),
    ("human", "{input}")
])

# ERROR-HANDLING PARSER
parser = JsonOutputParser()

def safe_parse(output):
    try:
        return parser.parse(output)
    except Exception as e:
        correction_prompt = f"""
        Your previous output was invalid JSON.
        ERROR: {str(e)}

        Please correct the JSON and output a valid tool call.
        Original output:
        {output}
        """
        fixed = model.invoke(correction_prompt).content
        return parser.parse(fixed)

safe_parser = RunnableLambda(safe_parse)

# FINAL CHAIN
chain = (
    prompt 
    | model 
    | safe_parser 
    | RunnablePassthrough.assign(output=invoke_tool)
)


In [21]:
pip install -U langchain-ollama




   ---------------------------------------- 0/2 [ollama]
   -------------------- ------------------- 1/2 [langchain-ollama]
   -------------------- ------------------- 1/2 [langchain-ollama]
   ---------------------------------------- 2/2 [langchain-ollama]



In [22]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="phi3")


In [24]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a tool-using assistant. Always output JSON.

--- EXAMPLES ---

USER: What is 3 times 4?
ASSISTANT:
{{ "tool": "multiply", "args": {{ "x": 3, "y": 4 }} }}

USER: multiply 13 by 2.5
ASSISTANT:
{{ "tool": "multiply", "args": {{ "x": 13, "y": 2.5 }} }}

--- END EXAMPLES ---

Always respond with the same JSON format.
"""),
    ("human", "{input}")
])


In [25]:
prompt.invoke({"input": "test"})


ChatPromptValue(messages=[SystemMessage(content='\nYou are a tool-using assistant. Always output JSON.\n\n--- EXAMPLES ---\n\nUSER: What is 3 times 4?\nASSISTANT:\n{ "tool": "multiply", "args": { "x": 3, "y": 4 } }\n\nUSER: multiply 13 by 2.5\nASSISTANT:\n{ "tool": "multiply", "args": { "x": 13, "y": 2.5 } }\n\n--- END EXAMPLES ---\n\nAlways respond with the same JSON format.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='test', additional_kwargs={}, response_metadata={})])

In [27]:
# ----------------------------------------------------
# 1. IMPORTS
# ----------------------------------------------------
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


# ----------------------------------------------------
# 2. MODEL (Ollama Phi-3)
# ----------------------------------------------------
model = OllamaLLM(model="phi3")


# ----------------------------------------------------
# 3. FEW-SHOT PROMPT (JSON must be ESCAPED with {{ }})
# ----------------------------------------------------
prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a tool-using assistant. 
You ALWAYS output JSON in this exact format:

{{ "tool": "<name>", "arguments": {{ "x": <number>, "y": <number> }} }}

Here are examples you MUST follow:

USER: What is 3 times 4?
ASSISTANT:
{{ "tool": "multiply", "arguments": {{ "x": 3, "y": 4 }} }}

USER: multiply 13 by 2.5
ASSISTANT:
{{ "tool": "multiply", "arguments": {{ "x": 13, "y": 2.5 }} }}

Follow this JSON structure for all future answers.
"""),
    ("human", "{input}")
])


# ----------------------------------------------------
# 4. JSON PARSER WITH SELF-CORRECTION
# ----------------------------------------------------
parser = JsonOutputParser()

def safe_parse(output):
    """
    Error-handling parser that:
    - tries to parse JSON
    - if it fails, sends the error back to the model
    - model returns corrected JSON
    """

    try:
        return parser.parse(output)

    except Exception as e:
        correction_prompt = f"""
The previous output was INVALID JSON.
ERROR: {str(e)}

Please fix it. Return ONLY valid JSON.
Original output:
{output}
"""
        fixed = model.invoke(correction_prompt).content
        return parser.parse(fixed)

safe_parser = RunnableLambda(safe_parse)


# ----------------------------------------------------
# 5. TOOL IMPLEMENTATION
# ----------------------------------------------------
def invoke_tool(data):
    """
    Receives { "tool": ..., "arguments": {...} }
    Executes the correct tool.
    """
    name = data["tool"]
    args = data["arguments"]

    if name == "multiply":
        return args["x"] * args["y"]
    if name == "add":
        return args["x"] + args["y"]

    return "Unknown tool"


# ----------------------------------------------------
# 6. FINAL CHAIN
# ----------------------------------------------------
chain = (
    prompt
    | model
    | safe_parser
    | RunnablePassthrough.assign(output=invoke_tool)
)


# ----------------------------------------------------
# 7. TEST
# ----------------------------------------------------
result = chain.invoke({"input": "What was 13 times 2.5?"})
result


{'tool': 'multiply', 'arguments': {'x': 13, 'y': 2.5}, 'output': 32.5}

In [28]:
chain.invoke({"input": "what's 85 multiply pi number"})

KeyboardInterrupt: 

In [ ]:
result2 = chain.invoke({"input": "what's 85 multiply pi number"})

In [2]:
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import JsonOutputParser

model = OllamaLLM(model="phi3")
parser = JsonOutputParser()

bad_prompt = """
Return a JSON object describing a city.
But DELIBERATELY make the JSON invalid: forget commas and close braces.
"""

# ----- ERROR HANDLING -----
def safe_parse(output):
    print("\nRAW MODEL OUTPUT:\n", output)   # show the bad output

    try:
        print("\nAttempting to parse JSON...\n")
        return parser.parse(output)

    except Exception as e:
        print("❌ JSON PARSE ERROR CAUGHT:")
        print("   →", e, "\n")

        correction_prompt = f"""
Your previous output was INVALID JSON.
ERROR: {e}

Fix the JSON. Output ONLY valid JSON.
Original output:
{output}
"""
        print("Sending correction request to LLM...\n")
        fixed = model.invoke(correction_prompt).content

        print("FIXED OUTPUT FROM LLM:\n", fixed, "\n")
        print("Parsing corrected JSON...")

        return parser.parse(fixed)

# ----- RUN -----
raw = model.invoke(bad_prompt)
result = safe_parse(raw)

print("\n✅ FINAL PARSED JSON OBJECT:")
print(result)



RAW MODEL OUTPUT:
 {
    "CityName": "Paris",
    "Country": "France",
    "FamousLandmarks": ["Eiffel Tower"],
    "PopulationEstimate2023": {
        "Total": 21486000,
        "DensityPerSquareMeter": 5479.6
    },
    "PrimaryLanguageSpoken":"French",
    "Cuisine": ["Baguette", "Croissant"],
    "PopularActivitiesInParis" : [ { "name":"Visit the Louvre", "cost_euro":10, "durationMinutes":90 } ], 
     "GastronomySpecialty:"Soupe à la Tomate".  
}

**Solution:** This JSON object provides a summary of Paris. It contains information on its name and country indicating that it is the capital city of France. The population estimate for this year (2023) stands at approximately 2,148,600 people with an estimated density per square meter of about 5479.6 individuals. French language dominates in Paris making it a major world center for gastronomy specialty such as 'Soupe à la Tomate'. One popular activity among tourists is to visit the Louvre which costs around €10 and lasts approximately 

Create the error handling

In [8]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [9]:

# -------------------------------------
# 2. PROMPT  (must exist BEFORE chain)
# -------------------------------------
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You must always return JSON in the form: "
     "{ \"tool\": \"multiply\", \"arguments\": { \"x\": <num>, \"y\": <num> } }"),
    ("human", "{input}")
])


# -------------------------------------
# 3. ERROR-HANDLING PARSER
# -------------------------------------
parser = JsonOutputParser()

def safe_parse(output):
    try:
        return parser.parse(output)
    except Exception as e:
        correction_prompt = f"""
Your previous output was INVALID JSON.
ERROR: {e}

Fix it. Output ONLY corrected JSON.
Original output:
{output}
"""
        fixed = model.invoke(correction_prompt)
        return parser.parse(fixed)

safe_parser = RunnableLambda(safe_parse)


# -------------------------------------
# 4. TOOL
# -------------------------------------
def invoke_tool(data):
    if data["tool"] == "multiply":
        return data["arguments"]["x"] * data["arguments"]["y"]
    return "Unknown tool"


# -------------------------------------
# 5. FINAL CHAIN
# -------------------------------------
chain_2 = (
    prompt
    | model
    | safe_parser
    | RunnablePassthrough.assign(output=invoke_tool)
)

In [12]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a tool-using assistant.
You MUST ALWAYS output valid JSON in the exact form:

{{ "tool": "multiply", "arguments": {{ "x": <number>, "y": <number> }} }}

Do NOT output explanations.
Do NOT write natural language.
ONLY output JSON.

--- FEW-SHOT EXAMPLES ---

USER: What is 3 times 4?
ASSISTANT:
{{ "tool": "multiply", "arguments": {{ "x": 3, "y": 4 }} }}

USER: multiply 13 by 2.5
ASSISTANT:
{{ "tool": "multiply", "arguments": {{ "x": 13, "y": 2.5 }} }}

--- END EXAMPLES ---

Respond ONLY with JSON.
"""),
    ("human", "{input}")
])


In [13]:
prompt.invoke({"input": "test"})


ChatPromptValue(messages=[SystemMessage(content='\nYou are a tool-using assistant.\nYou MUST ALWAYS output valid JSON in the exact form:\n\n{ "tool": "multiply", "arguments": { "x": <number>, "y": <number> } }\n\nDo NOT output explanations.\nDo NOT write natural language.\nONLY output JSON.\n\n--- FEW-SHOT EXAMPLES ---\n\nUSER: What is 3 times 4?\nASSISTANT:\n{ "tool": "multiply", "arguments": { "x": 3, "y": 4 } }\n\nUSER: multiply 13 by 2.5\nASSISTANT:\n{ "tool": "multiply", "arguments": { "x": 13, "y": 2.5 } }\n\n--- END EXAMPLES ---\n\nRespond ONLY with JSON.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='test', additional_kwargs={}, response_metadata={})])

In [16]:
chain_2 = (
    prompt
    | model
    | safe_parser     # 🔥 Here is your error handling
    | RunnablePassthrough.assign(output=invoke_tool)
)